- 保监会 预警模型 1 预处理

# 基本设置

In [10]:
import jieba
import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
# import pre
import os
from sqlalchemy import create_engine
from pandas.io import sql

import warnings
warnings.filterwarnings('ignore')

## 一些函数

In [2]:
def get_txt_encode(file_path):
    '''
    Function:
        获取txt文档的编码格式
    Arguments:
        file_path -> string -- 文档所在路径
    Return:
        encode -> string -- txt文档的编码格式
    '''
    decode_set = ['utf-8','gb18030','ISO-8859-2','gb2312','gbk', 'ANSI'] # 编码集
    encode = 0
    for k in decode_set:
        try :
            file = open(file_path, 'r', encoding = k).read()
            # print('\n\t读取成功： %s \n\t编码方式为： %s' %(file_path, k))
            encode = k
            break
        except :
            continue
    if encode:
        return encode
    else :
        return 'No encode'

In [3]:
def combined_data(data_folder, save_filename):
    '''
    保监会
        txt 文本：文件名（包含标题）
                  正  文（包含标题）
    '''
    file_list = os.listdir(data_folder)
    combined_data = []
    for file in file_list:
        tmp = os.getcwd() + '\\' + data_folder + '\\' + file
        encoding = get_txt_encode(tmp)
        f = open(tmp, "r+", encoding = encoding)
        ff = f.readlines()
        title = ff[0]  # 提取标题
        lines = ''
        for index, line in enumerate(ff):
            if index != 0:
                lines += line.strip()
        f.close()
        combined_data.append([title, lines])
        
    combined_data = pd.DataFrame(combined_data, columns = ['title', 'content'])
    combined_data.to_excel(save_filename)

In [4]:
def set_ch():
    '''
    功能：设定绘图时显示中文
    '''	
    from pylab import mpl
    mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
    mpl.rcParams['axes.unicode_minus'] = False   # 解决保存图像是负号'-'显示为方块的问题
set_ch()

# 导入数据

## 预警数据

In [5]:
folder_list = []

data_file_list = ['0612保监会预警数据.xlsx']

for folder_name in folder_list:
    data_folder = 'data/%s'%folder_name
    save_filename = 'data/%s.xlsx'%folder_name
    combined_data(data_folder, save_filename)  
    data_file_list.append(folder_name + '.xlsx')

In [8]:
cor_data_raw = pd.DataFrame()    
for file_name in data_file_list:
    tmp_data = pd.read_excel('data/20180622/%s'%file_name)
    print('%s: '%file_name, tmp_data.shape)
    cor_data_raw = pd.concat([cor_data_raw, tmp_data], axis = 0)

cor_data_raw['label'] = 1
print('未去空值时：', cor_data_raw.shape)
cor_data_raw = cor_data_raw.dropna(how = 'any')
print('已去空值时：', cor_data_raw.shape)
cor_data_raw.head()

0612保监会预警数据.xlsx:  (1628, 2)
未去空值时： (1628, 3)
已去空值时： (1628, 3)


,title,content,label
0,防风险治乱象 逾900张罚单直指险企问题重灾区,今日报纸每经网首页 > 今日报纸 > ...,1
1,调结构回保障“保险+科技”已成行业新发力点,今日报纸每经网首页 > 今日报纸 > ...,1
2,补短板强监管 2017年保监会密集发文60余份,今日报纸每经网首页 > 今日报纸 > ...,1
3,寿险公司开门红产品调查： 上市险企增保障范围 中小险企暗提收益,本报记者 苏向杲 险企开门红首月已过大半，寿险公司推出开门红的产品也浮出水面。...,1
4,渎职行为违规行为终身追责,本报北京1月17日电 （记者曲哲涵）保监会17日发布《打赢保险业防范化解重大风险攻坚战...,1


## 非预警数据

In [11]:
folder_list = ['0612保监会非预警',]

data_file_list = []

for folder_name in folder_list:
    data_folder = 'data/20180622/%s'%folder_name
    save_filename = 'data/%s.xlsx'%folder_name
    combined_data(data_folder, save_filename)  
    data_file_list.append(folder_name + '.xlsx')

In [12]:
uncor_data_raw = pd.DataFrame()    
for file_name in data_file_list:
    tmp_data = pd.read_excel('data/%s'%file_name)
    print('%s: '%file_name, tmp_data.shape)
    uncor_data_raw = pd.concat([uncor_data_raw, tmp_data], axis = 0)

uncor_data_raw['label'] = 0
print('未去空值时：', uncor_data_raw.shape)
uncor_data_raw = uncor_data_raw.dropna(how = 'any')
print('已去空值时：', uncor_data_raw.shape)
uncor_data_raw.head()

0612保监会非预警.xlsx:  (1924, 2)
未去空值时： (1924, 3)
已去空值时： (1923, 3)


,title,content,label
0,五星定位胆技巧，中奖的心态小技巧！\n,一招：咬定大奖加倍投注五星玩法和三星玩法都属于玩法，系出同门，从玩法规则上看，两种玩法相互关...,0
1,让你全面提升商业思维的20部电影\n,信息爆炸的时代里，我们会关注股市动荡、食品安全、转基因食物、鸿茅药酒事件、医疗体制、知识产权...,0
2,何为违法运用资金罪\n,说到是一个的话题，公司里面的资金不是可以随便乱用的，如果随便乱用就会构成违法运用现金罪，那么...,0
3,MFC易物点\n,很多人错误的理解，投资做MFC，一个人不找不卖货也赚钱就是骗局，是天上掉馅饼，其实这只是观念...,0
4,"深圳市平乐骨伤科医院违法涂改病历,伪造病历,拒绝住院病人查阅复制病历\n","深圳市平乐骨伤科医院违法涂改病历,伪造病历,拒绝住院病人查阅复制病历,矛盾正在激化！ 我友...",0


In [18]:
combined_data = pd.concat([cor_data_raw, uncor_data_raw], axis = 0)
print(combined_data.shape)
combined_data.head()

(3551, 3)


,title,content,label
0,防风险治乱象 逾900张罚单直指险企问题重灾区,今日报纸每经网首页 > 今日报纸 > ...,1
1,调结构回保障“保险+科技”已成行业新发力点,今日报纸每经网首页 > 今日报纸 > ...,1
2,补短板强监管 2017年保监会密集发文60余份,今日报纸每经网首页 > 今日报纸 > ...,1
3,寿险公司开门红产品调查： 上市险企增保障范围 中小险企暗提收益,本报记者 苏向杲 险企开门红首月已过大半，寿险公司推出开门红的产品也浮出水面。...,1
4,渎职行为违规行为终身追责,本报北京1月17日电 （记者曲哲涵）保监会17日发布《打赢保险业防范化解重大风险攻坚战...,1


In [19]:
combined_data.groupby('label')['title'].count()

label
0    1923
1    1628
Name: title, dtype: int64

In [20]:
combined_data.to_excel('data/20180621_circ_warn_data.xlsx', index = False)

# 预处理数据

In [11]:
# 预警数据
cor_data = cor_data_raw['content'].tolist()
print(len(cor_data))
corpus_cor = pre.handle_contents(cor_data)

save_filename = 'data/corpus_pre_warn_0621.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in corpus_cor:
    fid.write(data + '\n')
fid.close()

1628


In [12]:
# 非预警数据
uncor_data = uncor_data_raw['content'].tolist()
print(len(uncor_data))
corpus_uncor = pre.handle_contents(uncor_data)

save_filename = 'data/corpus_pre_no_warn_0621.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in corpus_uncor:
    fid.write(data + '\n')
fid.close()

1923
